# Лабораторная работа №5

## Задание:
1 . Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
#### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

#### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('property_assessment.csv')
print(Dataset.head())

         PID        CM_ID     GIS_ID  ST_NUM       ST_NAME         CITY  \
0  100018006  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
1  100018008  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
2  100019006  100019000.0  100019000   241.0  Lexington ST  EAST BOSTON   
3  100024002  100024000.0  100024000   249.0  Lexington ST  EAST BOSTON   
4  100041006  100041000.0  100041000   224.0  PRINCETON ST  EAST BOSTON   

   ZIP_CODE  BLDG_SEQ  NUM_BLDGS  LUC  ... FULL_BTH HLF_BTH KITCHENS TT_RMS  \
0    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   
1    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
2    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
3    2128.0         1          1  102  ...      1.0     0.0      1.0    5.0   
4    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   

     KITCHEN_TYPE           HEAT_TYPE         AC_TYPE FIREPLACES  NUM_PARK

### Создание искомого столбца

In [3]:
label_encoder = LabelEncoder()
Dataset['COND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])

### Нормализация данных

In [4]:
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
numeric_columns = Dataset.select_dtypes(include='number').columns.tolist()
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PID          10321 non-null  int64  
 1   CM_ID        10321 non-null  float64
 2   GIS_ID       10321 non-null  int64  
 3   ST_NUM       10321 non-null  float64
 4   ZIP_CODE     10321 non-null  float64
 5   BLDG_SEQ     10321 non-null  int64  
 6   NUM_BLDGS    10321 non-null  int64  
 7   LUC          10321 non-null  int64  
 8   RES_FLOOR    10321 non-null  float64
 9   GROSS_AREA   10321 non-null  float64
 10  LIVING_AREA  10321 non-null  float64
 11  LAND_VALUE   10321 non-null  int64  
 12  BLDG_VALUE   10321 non-null  int64  
 13  SFYI_VALUE   10321 non-null  int64  
 14  TOTAL_VALUE  10321 non-null  int64  
 15  YR_BUILT     10321 non-null  float64
 16  YR_REMODEL   10321 non-null  float64
 17  BED_RMS      10321 non-null  float64
 18  FULL_BTH     10321 non-null  float64
 19  HLF_

### Поиск лучших метрик

In [5]:
X = Dataset.drop(columns=['COND_ENC'])
y = Dataset['COND_ENC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Для решающего дерева
decision_tree = DecisionTreeClassifier()
dt_params = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_grid_search = GridSearchCV(decision_tree, dt_params, cv=5)
dt_grid_search.fit(X_train, y_train)

y_pred_dt = dt_grid_search.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree - Лучшее значение параметра: {dt_grid_search.best_params_}, Точность на тесте: {accuracy_dt}")

# Для случайного леса
random_forest = RandomForestClassifier()
rf_params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(random_forest, rf_params, cv=5)
rf_grid_search.fit(X_train, y_train)

y_pred_rf = rf_grid_search.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest - Лучшие значения параметров: {rf_grid_search.best_params_}, Точность на тесте: {accuracy_rf}")


Decision Tree - Лучшее значение параметра: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}, Точность на тесте: 0.8806006296924195
Random Forest - Лучшие значения параметров: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}, Точность на тесте: 0.9094211673528699


### Обучение на лучших параметрах

In [6]:
from sklearn.metrics import classification_report

# Для решающего дерева
best_dt_params = {'max_depth': 20}
best_decision_tree = DecisionTreeClassifier(**best_dt_params)
best_decision_tree.fit(X_train, y_train)

y_pred_best_dt = best_decision_tree.predict(X_test)
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)

print(f"Decision Tree с лучшими параметрами - Точность на тесте: {accuracy_best_dt}")
print("Classification Report для Decision Tree:")
print(classification_report(y_test, y_pred_best_dt))

# Для случайного леса
best_rf_params = {'max_depth': None, 'n_estimators': 50}
best_random_forest = RandomForestClassifier(**best_rf_params)
best_random_forest.fit(X_train, y_train)

y_pred_best_rf = best_random_forest.predict(X_test)
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)

print(f"Random Forest с лучшими параметрами - Точность на тесте: {accuracy_best_rf}")
print("Classification Report для Random Forest:")
print(classification_report(y_test, y_pred_best_rf))


Decision Tree с лучшими параметрами - Точность на тесте: 0.8723661903608622
Classification Report для Decision Tree:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3050
           1       0.72      0.82      0.77        62
           2       0.80      0.82      0.81        60
           3       1.00      1.00      1.00         1
           4       0.73      0.74      0.74       915
           5       0.40      0.54      0.46        41

    accuracy                           0.87      4129
   macro avg       0.76      0.81      0.78      4129
weighted avg       0.88      0.87      0.87      4129

Random Forest с лучшими параметрами - Точность на тесте: 0.9140227658028578
Classification Report для Random Forest:
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      3050
           1       0.88      0.73      0.80        62
           2       0.88      0.73      0.80        60
      